In [1]:
import pandas as pd
import streamlit as st
from typing import Any
import plotly as plo

In [2]:
datafile = pd.read_csv('C:\\Users\\savma\\dev_boilerplate_course\\dev_boilerplate_course\\src\\data\\run4moreProd.activities.csv')


In [3]:
activities_datafile = 'C:\\Users\\savma\\dev_boilerplate_course\\dev_boilerplate_course\\src\\data\\run4moreProd.activities.csv'

In [4]:
datafile.head(5)

,_id,distance,pace.hours,pace.minutes,pace.seconds,points,time.hours,time.minutes,time.seconds,user_id,createdAt,type
0,62873d1f181a6d5adf0f62b2,0.17,0,14,39,17,0,2,33,6206e062fb2d16fd06802520,2022-05-20T07:02:55.295Z,walk-run
1,62873d8e181a6d5adf0f62c2,4.10,0,13,19,410,0,54,41,6251c082520daa3cd653d214,2022-05-20T07:04:46.043Z,walk-run
2,62874060181a6d5adf0f62f0,0.15,0,17,32,15,0,2,40,6206e062fb2d16fd06802520,2022-05-20T07:16:48.452Z,walk-run
3,628740a3181a6d5adf0f62fd,0.00,0,0,0,0,0,0,59,6206e062fb2d16fd06802520,2022-05-20T07:17:55.980Z,walk-run
4,62874460181a6d5adf0f6318,0.54,0,10,2,54,0,5,29,6206e062fb2d16fd06802520,2022-05-20T07:33:52.419Z,walk-run


In [5]:
def csv_from_mongo_to_dfs(datafile: str) -> tuple[Any, pd.DataFrame]:
    df = pd.read_csv(datafile)
    assert isinstance(pd.to_datetime(df["createdAt"]).dt.date, object)
    df["date"] = pd.to_datetime(df["createdAt"]).dt.date
    df = df.drop("_id", axis=1)
    # place date column as first column
    date = df.pop("date")
    df.insert(loc=0, column="date", value=date)
    df_acts = df.groupby("date").size().to_frame(name="activities").reset_index()
    df_users = df.groupby("date")["user_id"].nunique().reset_index()
    df_users.rename(columns={"user_id": "users"}, inplace=True)
    return df_acts, df_users
    

In [6]:
def date_slider(df: pd.DataFrame) -> tuple:
    slider_dates = st.sidebar.slider(
        "# Pick a date range",
        df.date.min(),
        df.date.max(),
        (df.date.min(), df.date.max()),
    )
    return slider_dates

In [7]:
def update_df_dates(df: pd.DataFrame, slider_dates: tuple) -> pd.DataFrame:
    date_range = (df.date >= slider_dates[0]) & (df.date <= slider_dates[1])
    filter_df_dates = df.loc[date_range]
    return filter_df_dates

In [8]:
def plot_daily(df: pd.DataFrame) -> st.plotly_chart:
    # streamlit with plotly chart
    fig = px.line(df, x="date", y=df.columns[1])
    st.plotly_chart(fig)

In [9]:
def plot_daily_dashboard(datafile: str):
    df_acts, df_users = csv_from_mongo_to_dfs(datafile)
    slider_dates = date_slider(df_acts)
    st.header(df_acts.columns[1].capitalize())
    df_acts = update_df_dates(df_acts, slider_dates)
    plot_daily(df_acts)
    st.header(df_users.columns[1].capitalize())
    df_users = update_df_dates(df_users, slider_dates)
    plot_daily(df_users)